# Missing Values Exercises


## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/ids720_specific/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_missing.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:


```python
assert set(results.keys()) == {
    "ex2_avg_income",
    "ex3_share_making_9999999",
    "ex3_share_making_zero",
    "ex5_avg_income",
    "ex8_avg_income_black",
    "ex8_avg_income_white",
    "ex8_racial_difference",
    "ex9_avg_income_black",
    "ex9_avg_income_white",
    "ex10_wage_gap",
}
```

### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.


## Exercises

### Exercise 1

Today, we will be using the ACS data we used during out first `pandas` exercise to examine the US income distribution, and how it varies by race. Note that because the US income distribution has a very small number of people with *extremely* high incomes, and the ACS is just a sample of Americans, the far right tail of the distribution will not be very well estimated. However, this data should suffice for helping to understand wealth inequality in the United States. 

To begin, load the ACS Data we used in our first pandas exercise. That [data can be found here](https://github.com/nickeubank/MIDS_Data/tree/master/US_AmericanCommunitySurvey). We'll be working with `US_ACS_2017_10pct_sample.dta`. 

In [41]:
import pandas as pd

pd.set_option("mode.copy_on_write", True)

url = "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"

df = pd.read_stata(url)

df.sample(5)

,year,datanum,serial,cbserial,numprec,subsamp,hhwt,hhtype,cluster,adjust,...,migcounty1,migmet131,vetdisab,diffrem,diffphys,diffmob,diffcare,diffsens,diffeye,diffhear
244927,2017,1,263331,2.017001e+12,4,36,75,"female householder, no husband present",2.017003e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
136667,2017,1,1225819,2.017001e+12,2,73,198,married-couple family household,2.017012e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
633,2017,1,1077974,2.017001e+12,4,16,77,married-couple family household,2.017011e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
188801,2017,1,1142723,2.017000e+12,1 person record,24,46,"female householder, living alone",2.017011e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
195537,2017,1,252634,2.017001e+12,3,22,68,married-couple family household,2.017003e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no


### Exercise 2

Let's begin by calculating the mean US incomes from this data (recall that income is stored in the `inctot` variable). Store the answer in `results` under the key `"ex2_avg_income"`.

In [42]:
results = {}

results["ex2_avg_income"] = df["inctot"].mean()

print(f"The mean U.S. income is ${results['ex2_avg_income']:,.2f}")

The mean U.S. income is $1,723,646.27


### Exercise 3

Hmmm... That doesn't look right. The average American is definitely not earning that much a year! Let's look at the values of `inctot` using `value_counts()`. Do you see a problem?

Now use `value_counts()` with the argument `normalize=True` to see proportions of the sample that report each value instead of the count of people in each category. What percentage of our sample has an income of 9,999,999? Store that proportion (between 0 and 1) as `"ex3_share_making_9999999"`. What percentage has an income of 0? Store that proportion as `"ex3_share_making_zero"`.

(Recall `.value_counts()` returns a Series, so you can pull values out with our usual pandas tools.)

In [43]:
df["inctot"].value_counts(dropna=False)

inctot
9999999    53901
0          33679
30000       4778
50000       4414
40000       4413
           ...  
70520          1
76680          1
57760          1
200310         1
505400         1
Name: count, Length: 8471, dtype: int64

In [44]:
inc_count = df["inctot"].value_counts(normalize=True)
inc_count

inctot
9999999    0.168967
0          0.105575
30000      0.014978
50000      0.013837
40000      0.013834
             ...   
70520      0.000003
76680      0.000003
57760      0.000003
200310     0.000003
505400     0.000003
Name: proportion, Length: 8471, dtype: float64

In [45]:
results["ex3_share_making_9999999"] = inc_count[9999999]
print(
    f"The share of people making $9,999,999 is {results['ex3_share_making_9999999']:.2%}"
)

results["ex3_share_making_zero"] = inc_count[0]
print(f"The share of people making $0 is {results['ex3_share_making_zero']:.2%}")

The share of people making $9,999,999 is 16.90%
The share of people making $0 is 10.56%


### Exercise 4

As we discussed before, the ACS uses a value of 9999999 to denote that income information is not available for someone. The problem with using this kind of "sentinel value" is that pandas doesn't understand that this is supposed to denote missing data, and so when it averages the variable, it doesn't know to ignore 9999999. 

To help out `pandas`, use the `replace` command to replace all values of 9999999 with `np.nan`. 

In [46]:
import numpy as np

# replace 9999999 with NaN
df["inctot"] = df["inctot"].replace(9999999, np.nan)

### Exercise 5

Now that we've properly labeled our missing data as `np.nan`, let's calculate the average US income once more. Store the answer in `results` under the key `"ex5_avg_income"`.

In [47]:
results["ex5_avg_income"] = df["inctot"].mean()
print(f"The mean U.S. income is ${results['ex5_avg_income']:,.2f}")

The mean U.S. income is $40,890.18


### Exercise 6

OK, now we've been able to get a reasonable average income number. As we can see, a major advantage of using `np.nan` is that `pandas` knows that `np.nan` observations should just be ignored when we are calculating means. 

But it's not enough to just get rid of the people who had `inctot` values of 9999999. We also need to know why those values were missing. Suppose, for example, that the value of 9999999 was used for anyone who made more than 100,000 dollars: if we just dropped those people, then our estimate of average income wouldn't mean much, would it?

So let's make sure we understand *why* data is missing for some people. If you recall from our last exercise, it seemed to be the case that most of the people who had incomes of 9999999 were children. Let's make sure that's true by looking at the distribution of the variable `age` for people for whom `inctot` is missing (i.e. subset the data to people with `inctot` missing, then look at the values of `age` with `value_counts()`).

Then do the opposite: look at the distribution of the `age` variable for people who whom `inctot` is *not* missing. 

Can you determine when 9999999 was being used? Is it ok we're excluding those people from our analysis?

Note: In this data, Python doesn't understand `age` is a number; it thinks it is a string because the original data has categories like "90 (90+ in 1980 and 1990)" and "less than 1 year old". So you can't just use `min()` or `max()`. We'll discuss converting string variables into numbers in a future class.

In [48]:
# look at the distribution of the age variable for people who whom `inctot` is missing
df["age"][df["inctot"].isna()].value_counts(dropna=False)

age
10    3997
9     3977
14    3847
12    3845
13    3800
      ... 
39       0
38       0
37       0
36       0
96       0
Name: count, Length: 97, dtype: int64

In [49]:
# look at the distribution of the age variable for people who whom inctot is NOT missing

df["age"][df["inctot"].notna()].value_counts(dropna=False)

age
60                      4950
54                      4821
59                      4776
56                      4776
58                      4734
                        ... 
5                          0
4                          0
3                          0
2                          0
less than 1 year old       0
Name: count, Length: 97, dtype: int64

**Most people whose `inctot` value is 9999999 appear to be children, and those that do not have 9999999 value appear to not be children. Therefore, I think it's okay to exclude these people from analysis.**

### Exercise 7

Great, so now we know why those people had missing data, and we're ok with excluding them. 

But as we previously noted, there are also a lot of observations of zero income in our data, and it's not clear that we want everyone with a zero-income *should* be included in this average, since those may be people who are retired, or in school. 

Let's limit our attention to people who are currently working by subsetting to only employed respondents. We can do this using `empstat`. Remember you can use `value_counts()` to see what values of `empstat` are in the data!

In [50]:
df["empstat"].value_counts(dropna=False)

empstat
employed              148758
not in labor force    104676
n/a                    57843
unemployed              7727
Name: count, dtype: int64

In [51]:
df_employed = df[df["empstat"] == "employed"]

# look at the distribution of the age variable for people who are employed
df_employed["age"].value_counts(dropna=False)

age
54                      3591
53                      3445
56                      3422
55                      3415
47                      3378
                        ... 
5                          0
4                          0
3                          0
2                          0
less than 1 year old       0
Name: count, Length: 97, dtype: int64

### Exercise 8

Now let's estimate the racial income gap in the United States. What is the average salary for employed Black Americans, and what is the average salary for employed White Americans? In percentage terms, how much more does the average White American make than the average Black American?

**Note:** these values are not quite accurate estimates. As we'll discuss in later lessons, to get completely accurate estimates from the ACS we have to take into account how people were selected to be interviewed. But you get pretty good estimates in most cases even without weights—your estimate of the racial wage gap without weights is within 5\% of the corrected value. 

**Note:** This is actually an underestimate of the wage gap. The US Census treats Hispanic respondents as a sub-category of "White." While all ethnic distinctions are socially constructed, and so on some level these distinctions are all deeply problematic, this coding is inconsistent with what most Americans think of when they hear the term "White," a term *most* Americans think of as a category that is mutually exclusive of being Hispanic or Latino (categories which are also usually conflated in American popular discussion). With that in mind, most researchers working with US Census data split "White" into "White, Hispanic" and "White, Non-Hispanic" using `race` *and* `hispan`. But for the moment, just identify "White" respondents using the value in `race`.

Store your results in `results` under the keys `"ex8_avg_income_black"`, `"ex8_avg_income_white"`, and the percentage difference as `ex8_racial_difference`. Please note the wording above when calculating the percentage difference to ensure you get the reference category correct, and interpret your result as well.

In [52]:
# look at the distribution of race for people who are employed
df_employed["race"].value_counts(dropna=False)

race
white                               116017
black/african american/negro         13175
other asian or pacific islander       6424
other race, nec                       5755
two major races                       3135
chinese                               2149
american indian or alaska native      1290
three or more major races              426
japanese                               387
Name: count, dtype: int64

In [53]:
black_income = df_employed[df_employed["race"] == "black/african american/negro"][
    "inctot"
]
white_income = df_employed[df_employed["race"] == "white"]["inctot"]

results["ex8_avg_income_black"] = black_income.mean()
results["ex8_avg_income_white"] = white_income.mean()

print(
    f"The average income for black employed population is ${results['ex8_avg_income_black']:,.2f}"
    + "\n"
    + f"The average income for white employed population is ${results['ex8_avg_income_white']:,.2f}"
)

The average income for black employed population is $41,747.95
The average income for white employed population is $60,473.15


In [54]:
results["ex8_racial_difference"] = (
    (results["ex8_avg_income_white"] - results["ex8_avg_income_black"])
    / results["ex8_avg_income_black"]
    * 100
)
print(
    f"The percentage difference in income between black and white employed population is {results['ex8_racial_difference']/100:.2%}"
)

The percentage difference in income between black and white employed population is 44.85%


### Exercise 9


As noted above, these estimates are not actually *quite* correct because we aren't taking into account the fact that when the US Census decided who to survey, not all Americans had the same likelihood of being asked. The US American Community Survey is an example of a *weighted* survey (essentially, people from smaller subpopulations have a higher likelihood of being included to ensure enough individuals in the final survey to constitute a representative sample that can be used statistically). 

To calculate a weighted average that takes into account these survey weights (in other words, a more accurate estimate of US incomes), you need to use the following formula:

$$weighted\_mean\_of\_x = \frac{\sum_i x_i * weight_i}{\sum_i weight_i}$$

(As you can see, when $weight_i$ is constant for all observations, this just simplifies to our normal formula for mean values. It is only when weights vary across individuals that weights must be explicitly addressed).

In this data, weights are stored in the variable `perwt`, which is the number of people for which each observation is a stand-in (the inverse of that observation's sampling probability). 

Using the formula, re-calculate the *weighted* average income for both populations and store them as `ex9_avg_income_white` and `ex9_avg_income_black`.


In [55]:
df_employed["perwt"].value_counts(dropna=False)

perwt
66     1543
63     1537
62     1528
59     1503
64     1490
       ... 
664       1
833       1
997       1
528       1
640       1
Name: count, Length: 780, dtype: int64

In [56]:
black_weight = df_employed[df_employed["race"] == "black/african american/negro"][
    "perwt"
]
black_weighted_inc = black_income * black_weight
results["ex9_avg_income_black"] = black_weighted_inc.sum() / black_weight.sum()

print(
    f"The weighted average income for black employed population is ${results['ex9_avg_income_black']:,.2f}"
)

The weighted average income for black employed population is $40,430.95


In [57]:
white_weight = df_employed[df_employed["race"] == "white"]["perwt"]
white_weighted_inc = white_income * white_weight
results["ex9_avg_income_white"] = white_weighted_inc.sum() / white_weight.sum()

print(
    f"The weighted average income for white employed population is ${results['ex9_avg_income_white']:,.2f}"
)

The weighted average income for white employed population is $58,361.48


### Exercise 10

Now calculate the weighted average income gap between *non-Hispanic* White Americans and Black Americans. What percentage more do employed White non-Hispanic Americans earn than employed Black Americans? Store as `"ex10_wage_gap"`.

In [58]:
df_employed["hispan"].value_counts(dropna=False)

hispan
not hispanic    128293
mexican          12437
other             5302
puerto rican      1787
cuban              939
Name: count, dtype: int64

In [59]:
white_nonhis_weight = df_employed[df_employed["race"] == "white"][
    df_employed["hispan"] == "not hispanic"
]["perwt"]
white_nonhis_income = df_employed[df_employed["race"] == "white"][
    df_employed["hispan"] == "not hispanic"
]["inctot"]
white_weighted_inc = white_nonhis_income * white_nonhis_weight
nonhis_white = white_weighted_inc.sum() / white_nonhis_weight.sum()

print(
    f"The weighted average income for white employed population is ${results['ex9_avg_income_white']:,.2f}"
)

The weighted average income for white employed population is $58,361.48


/var/folders/rs/c8pwrc0n0w5dp0btv3ttdpfw0000gn/T/ipykernel_78823/2083852603.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  white_nonhis_weight = df_employed[df_employed["race"] == "white"][
/var/folders/rs/c8pwrc0n0w5dp0btv3ttdpfw0000gn/T/ipykernel_78823/2083852603.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  white_nonhis_income = df_employed[df_employed["race"] == "white"][


In [60]:
results["ex10_wage_gap"] = (
    (nonhis_white - results["ex9_avg_income_black"])
    / results["ex9_avg_income_black"]
    * 100
)
print(
    f"The percentage difference in income between black and white employed population is {results['ex10_wage_gap']/100:.2%}"
)

The percentage difference in income between black and white employed population is 52.53%


In [61]:
for k, v in results.items():
    print(f"{k}: {v}")

ex2_avg_income: 1723646.2703978634
ex3_share_making_9999999: 0.1689665333350052
ex3_share_making_zero: 0.10557547867738336
ex5_avg_income: 40890.177564946454
ex8_avg_income_black: 41747.949905123336
ex8_avg_income_white: 60473.15372747098
ex8_racial_difference: 44.85299006275197
ex9_avg_income_black: 40430.953355310274
ex9_avg_income_white: 58361.48196061399
ex10_wage_gap: 52.52989147705372


### Exercise 11

Is that greater or less than the difference you found in Exercise 8? Why do you think that's the case?

**The difference is greater than that in exercise 8 - an increase from 44.85% to 52.53%, which likely occurs due to**\
**- The removal of the employed hispanic population from white population.**\
**- The initial non-weighted averaging method gives minorities, who often have less income on average, more representation than their actual proportion.**

In [62]:
assert set(results.keys()) == {
    "ex2_avg_income",
    "ex3_share_making_9999999",
    "ex3_share_making_zero",
    "ex5_avg_income",
    "ex8_avg_income_black",
    "ex8_avg_income_white",
    "ex8_racial_difference",
    "ex9_avg_income_black",
    "ex9_avg_income_white",
    "ex10_wage_gap",
}